In [2]:
#import necessary libraries
import pandas as pd
import numpy as np

In [6]:
#Get Data
cgm_data = pd.read_csv("Project 1 Student Files New/CGMData.csv",low_memory=False)
insulin_data = pd.read_csv("Project 1 Student Files New/InsulinData.csv",low_memory=False)

#date_to_remove=cgm_data_csv[cgm_data_csv['Sensor Glucose (mg/dL)'].isna()]['Date'].unique()
#cgm_data_csv=cgm_data_csv.set_index('Date').drop(index=date_to_remove).reset_index()
#cgm_data=cgm_data_csv.copy()

#Arranging in acsending order and having TimeStamp
cgm_data["TimeStamp"] = cgm_data[["Date", "Time"]].apply(" ".join, axis=1)
cgm_data["TimeStamp"] = pd.to_datetime(cgm_data['TimeStamp'])
cgm_data_final=cgm_data.sort_values(by =['TimeStamp'])

insulin_data["TimeStamp"] = insulin_data[["Date", "Time"]].apply(" ".join, axis=1)
insulin_data["TimeStamp"] = pd.to_datetime(insulin_data['TimeStamp'])
insulin_data_final=insulin_data.sort_values(by =['TimeStamp'])

#Finding start of Auto mode in Insulin Data
#auto_mode_starting_indices = insulin_data_final.index[insulin_data_final['Alarm'] == "AUTO MODE ACTIVE PLGM OFF"].tolist()
auto_mode_starting_index_ins = np.where(insulin_data_final['Alarm'] == "AUTO MODE ACTIVE PLGM OFF")[0][0]
auto_mode_starting_index_ins_row_values = insulin_data_final.iloc[auto_mode_starting_index_ins]

 
#Finding the corresponding auto mode in CGM Data
for i in range(len(cgm_data_final)):
 #if (cgm_data.iloc[i]['Date'] == auto_mode_starting_index_ins_row_values.iloc[0]['Date']) & (cgm_data.iloc[i]['Time'] > auto_mode_starting_index_ins_row_values.iloc[0]['Time']):
  if cgm_data_final.iloc[i]['TimeStamp'] > auto_mode_starting_index_ins_row_values['TimeStamp']:
        auto_mode_starting_index_cgm = i
        break;
auto_mode_starting_index_cgm_row_values = cgm_data_final.iloc[auto_mode_starting_index_cgm]

#Filtering Rows by leaving out data without enough samples
dates=[]
manual_mode_cgm_df = cgm_data_final[0:auto_mode_starting_index_cgm-1] 
auto_mode_cgm_df = cgm_data_final[auto_mode_starting_index_cgm:]

auto_dates = auto_mode_cgm_df.groupby('Date')['Sensor Glucose (mg/dL)'].count().dropna().index.tolist()
cgm_auto_df_final = auto_mode_cgm_df.loc[auto_mode_cgm_df['Date'].isin(auto_dates)]

manual_dates = manual_mode_cgm_df.groupby('Date')['Sensor Glucose (mg/dL)'].count().dropna().index.tolist()
cgm_manual_df_final = manual_mode_cgm_df.loc[manual_mode_cgm_df['Date'].isin(manual_dates)]

#Fixing timestamp as index as it is necessary for between_time fun
cgm_auto_df_final = cgm_auto_df_final.set_index(cgm_auto_df_final['TimeStamp'])
cgm_manual_df_final = cgm_manual_df_final.set_index(cgm_manual_df_final['TimeStamp'])

#Final Dataframes for overnight,daytime and whole day of CGM Data Auto mode
overnight_auto_df = cgm_auto_df_final.between_time('00:00:00','05:59:59')
daytime_auto_df = cgm_auto_df_final.between_time('06:00:00','23:59:59')
wholeday_auto_df = cgm_auto_df_final.between_time('00:00:00','23:59:59')

#Final Dataframes for overnight,daytime and whole day of CGM Data Manual mode
overnight_manual_df = cgm_manual_df_final.between_time('00:00:00','05:59:59')
daytime_manual_df = cgm_manual_df_final.between_time('06:00:00','23:59:59')
wholeday_manual_df = cgm_manual_df_final.between_time('00:00:00','23:59:59')


In [7]:
# Calculate the required percentages

# manual overnight
manual_overnight_hyperglycemia = overnight_manual_df.loc[overnight_manual_df['Sensor Glucose (mg/dL)']>180].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
manual_overnight_hyperglycemia_critical = overnight_manual_df.loc[overnight_manual_df['Sensor Glucose (mg/dL)']>250].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
manual_overnight_in_range = overnight_manual_df.loc[(overnight_manual_df['Sensor Glucose (mg/dL)']>=70)&(overnight_manual_df['Sensor Glucose (mg/dL)']<=180)].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
manual_overnight_in_range_secondary = overnight_manual_df.loc[(overnight_manual_df['Sensor Glucose (mg/dL)']>=70)&(overnight_manual_df['Sensor Glucose (mg/dL)']<=150)].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
manual_overnight_hypoglycemia_lvl1 = overnight_manual_df.loc[overnight_manual_df['Sensor Glucose (mg/dL)']<70].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
manual_overnight_hypoglycemia_lvl2 = overnight_manual_df.loc[overnight_manual_df['Sensor Glucose (mg/dL)']<54].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100

#auto overnight
auto_overnight_hyperglycemia = overnight_auto_df.loc[overnight_auto_df['Sensor Glucose (mg/dL)']>180].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
auto_overnight_hyperglycemia_critical = overnight_auto_df.loc[overnight_auto_df['Sensor Glucose (mg/dL)']>250].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
auto_overnight_in_range = overnight_auto_df.loc[(overnight_auto_df['Sensor Glucose (mg/dL)']>=70)&(overnight_auto_df['Sensor Glucose (mg/dL)']<=180)].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
auto_overnight_in_range_secondary = overnight_auto_df.loc[(overnight_auto_df['Sensor Glucose (mg/dL)']>=70)&(overnight_auto_df['Sensor Glucose (mg/dL)']<=150)].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
auto_overnight_hypoglycemia_lvl1 = overnight_auto_df.loc[overnight_auto_df['Sensor Glucose (mg/dL)']<70].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
auto_overnight_hypoglycemia_lvl2 = overnight_auto_df.loc[overnight_auto_df['Sensor Glucose (mg/dL)']<54].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100


# manual daytime
manual_daytime_hyperglycemia = daytime_manual_df.loc[daytime_manual_df['Sensor Glucose (mg/dL)']>180].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
manual_daytime_hyperglycemia_critical = daytime_manual_df.loc[daytime_manual_df['Sensor Glucose (mg/dL)']>250].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
manual_daytime_in_range = daytime_manual_df.loc[(daytime_manual_df['Sensor Glucose (mg/dL)']>=70)&(daytime_manual_df['Sensor Glucose (mg/dL)']<=180)].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
manual_daytime_in_range_secondary = daytime_manual_df.loc[(daytime_manual_df['Sensor Glucose (mg/dL)']>=70)&(daytime_manual_df['Sensor Glucose (mg/dL)']<=150)].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
manual_daytime_hypoglycemia_lvl1 = daytime_manual_df.loc[daytime_manual_df['Sensor Glucose (mg/dL)']<70].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
manual_daytime_hypoglycemia_lvl2 = daytime_manual_df.loc[daytime_manual_df['Sensor Glucose (mg/dL)']<54].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100

#auto daytime
auto_daytime_hyperglycemia = daytime_auto_df.loc[daytime_auto_df['Sensor Glucose (mg/dL)']>180].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
auto_daytime_hyperglycemia_critical = daytime_auto_df.loc[daytime_auto_df['Sensor Glucose (mg/dL)']>250].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
auto_daytime_in_range = daytime_auto_df.loc[(daytime_auto_df['Sensor Glucose (mg/dL)']>=70)&(daytime_auto_df['Sensor Glucose (mg/dL)']<=180)].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
auto_daytime_in_range_secondary = daytime_auto_df.loc[(daytime_auto_df['Sensor Glucose (mg/dL)']>=70)&(daytime_auto_df['Sensor Glucose (mg/dL)']<=150)].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
auto_daytime_hypoglycemia_lvl1 = daytime_auto_df.loc[daytime_auto_df['Sensor Glucose (mg/dL)']<70].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
auto_daytime_hypoglycemia_lvl2 = daytime_auto_df.loc[daytime_auto_df['Sensor Glucose (mg/dL)']<54].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100

# manual wholeday
manual_wholeday_hyperglycemia = wholeday_manual_df.loc[wholeday_manual_df['Sensor Glucose (mg/dL)']>180].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
manual_wholeday_hyperglycemia_critical = wholeday_manual_df.loc[wholeday_manual_df['Sensor Glucose (mg/dL)']>250].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
manual_wholeday_in_range = wholeday_manual_df.loc[(wholeday_manual_df['Sensor Glucose (mg/dL)']>=70)&(wholeday_manual_df['Sensor Glucose (mg/dL)']<=180)].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
manual_wholeday_in_range_secondary = wholeday_manual_df.loc[(wholeday_manual_df['Sensor Glucose (mg/dL)']>=70)&(wholeday_manual_df['Sensor Glucose (mg/dL)']<=150)].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
manual_wholeday_hypoglycemia_lvl1 = wholeday_manual_df.loc[wholeday_manual_df['Sensor Glucose (mg/dL)']<70].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
manual_wholeday_hypoglycemia_lvl2 = wholeday_manual_df.loc[wholeday_manual_df['Sensor Glucose (mg/dL)']<54].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100

#auto wholeday
auto_wholeday_hyperglycemia = wholeday_auto_df.loc[wholeday_auto_df['Sensor Glucose (mg/dL)']>180].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
auto_wholeday_hyperglycemia_critical = wholeday_auto_df.loc[wholeday_auto_df['Sensor Glucose (mg/dL)']>250].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
auto_wholeday_in_range = wholeday_auto_df.loc[(wholeday_auto_df['Sensor Glucose (mg/dL)']>=70)&(wholeday_auto_df['Sensor Glucose (mg/dL)']<=180)].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
auto_wholeday_in_range_secondary = wholeday_auto_df.loc[(wholeday_auto_df['Sensor Glucose (mg/dL)']>=70)&(wholeday_auto_df['Sensor Glucose (mg/dL)']<=150)].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
auto_wholeday_hypoglycemia_lvl1 = wholeday_auto_df.loc[wholeday_auto_df['Sensor Glucose (mg/dL)']<70].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100
auto_wholeday_hypoglycemia_lvl2 = wholeday_auto_df.loc[wholeday_auto_df['Sensor Glucose (mg/dL)']<54].groupby('Date')['Sensor Glucose (mg/dL)'].count()/288*100



result_dict = {}
manual_dates=(cgm_manual_df_final['Date']).nunique()

auto_dates=(cgm_auto_df_final['Date']).nunique()

result_dict['overnight_hyperglycemia']= [(manual_overnight_hyperglycemia.sum()/manual_dates),(auto_overnight_hyperglycemia.sum()/auto_dates)]
result_dict['overnight_hyperglycemia_critical']= [(manual_overnight_hyperglycemia_critical.sum()/manual_dates),(auto_overnight_hyperglycemia_critical.sum()/auto_dates)]
result_dict['overnight_in_range']= [(manual_overnight_in_range.sum()/manual_dates),(auto_overnight_in_range.sum()/auto_dates)]
result_dict['overnight_in_range_secondary'] = [(manual_overnight_in_range_secondary.sum()/manual_dates),(auto_overnight_in_range_secondary.sum()/auto_dates)]
result_dict['overnight_hypoglycemia_lvl1'] = [(manual_overnight_hypoglycemia_lvl1.sum()/manual_dates),(auto_overnight_hypoglycemia_lvl1.sum()/auto_dates)]
result_dict['overnight_hypoglycemia_lvl2'] = [(np.nan_to_num(manual_overnight_hypoglycemia_lvl2.sum())/manual_dates),(np.nan_to_num(auto_overnight_hypoglycemia_lvl2.sum())/auto_dates)]

result_dict['daytime_hyperglycemia']= [(manual_daytime_hyperglycemia.sum()/manual_dates),(auto_daytime_hyperglycemia.sum()/auto_dates)]
result_dict['daytime_hyperglycemia_critical']= [(manual_daytime_hyperglycemia_critical.sum()/manual_dates),(auto_daytime_hyperglycemia_critical.sum()/auto_dates)]
result_dict['daytime_in_range']= [(manual_daytime_in_range.sum()/manual_dates),(auto_daytime_in_range.sum()/auto_dates)]
result_dict['daytime_in_range_secondary'] = [(manual_daytime_in_range_secondary.sum()/manual_dates),(auto_daytime_in_range_secondary.sum()/auto_dates)]
result_dict['daytime_hypoglycemia_lvl1'] = [(manual_daytime_hypoglycemia_lvl1.sum()/manual_dates),(auto_daytime_hypoglycemia_lvl1.sum()/auto_dates)]
result_dict['daytime_hypoglycemia_lvl2'] = [(manual_daytime_hypoglycemia_lvl2.sum()/manual_dates),(auto_daytime_hypoglycemia_lvl2.sum()/auto_dates)]

result_dict['wholeday_hyperglycemia']= [(manual_wholeday_hyperglycemia.sum()/manual_dates),(auto_wholeday_hyperglycemia.sum()/auto_dates)]
result_dict['wholeday_hyperglycemia_critical']= [(manual_wholeday_hyperglycemia_critical.sum()/manual_dates),(auto_wholeday_hyperglycemia_critical.sum()/auto_dates)]
result_dict['wholeday_in_range']= [(manual_wholeday_in_range.sum()/manual_dates),(auto_wholeday_in_range.sum()/auto_dates)]
result_dict['wholeday_in_range_secondary'] = [(manual_wholeday_in_range_secondary.sum()/manual_dates),(auto_wholeday_in_range_secondary.sum()/auto_dates)]
result_dict['wholeday_hypoglycemia_lvl1'] = [(manual_wholeday_hypoglycemia_lvl1.sum()/manual_dates),(auto_wholeday_hypoglycemia_lvl1.sum()/auto_dates)]
result_dict['wholeday_hypoglycemia_lvl2'] = [(manual_wholeday_hypoglycemia_lvl2.sum()/manual_dates),(auto_wholeday_hypoglycemia_lvl2.sum()/auto_dates)]

resultDataframe = pd.DataFrame(result_dict)

resultDataframe.to_csv("Result.csv",index=False,header=False)

In [66]:
resultDataframe

#result_dict['overnight_hypoglycemia_lvl2']
#auto_daytime_hypoglycemia_lvl2.mean(axis=0)

#['Sensor Glucose (mg/dL)'].count()/288*100

#dates=list(cgm_data_final['Date'].value_counts().where(lambda x:x>=288).index)
#.where(lambda x:x>0.8*288)
#.where(lambda x:x>0.8*288)
#manual_mode_cgm_df


#print(np.where(insulin_data_final['Alarm'] == "AUTO MODE ACTIVE PLGM OFF")[0][0])

#auto_mode_starting_indices
#auto_mode_starting_index_ins_row_values

#auto_mode_starting_index_cgm_row_values

#print(insulin_data_final[insulin_data_final['Alarm'] == "AUTO MODE ACTIVE PLGM OFF"]['TimeStamp'])
#insulin_data_final.iloc[1304]


#cgm_data[["TimeStamp"]]
#cgm_data.iloc[4256]
#auto_mode_starting_index_ins_row_values.iloc[0]['TimeStamp']
#cgm_data
#insulin_data
#cgm_data.iloc[0]['TimeStamp']
#auto_mode_starting_index_ins_row_values.iloc[0]['Date']
#cgm_data.iloc[4256]['Date']
#cgm_data.iloc[4256]['Time']
#auto_mode_starting_index_ins_row_values.iloc[0]['TimeStamp']

,overnight_hyperglycemia,overnight_hyperglycemia_critical,overnight_in_range,overnight_in_range_secondary,overnight_hypoglycemia_lvl1,overnight_hypoglycemia_lvl2,daytime_hyperglycemia,daytime_hyperglycemia_critical,daytime_in_range,daytime_in_range_secondary,daytime_hypoglycemia_lvl1,daytime_hypoglycemia_lvl2,wholeday_hyperglycemia,wholeday_hyperglycemia_critical,wholeday_in_range,wholeday_in_range_secondary,wholeday_hypoglycemia_lvl1,wholeday_hypoglycemia_lvl2
0,6.875000,2.013889,17.847222,12.430556,0.277778,0.0,20.277778,6.875000,37.638889,29.097222,3.819444,1.736111,27.152778,8.888889,55.486111,41.527778,4.097222,1.736111
1,2.577862,0.431397,20.486111,17.297980,0.420875,0.0,19.728535,5.208333,49.379209,36.637205,3.524832,0.641835,22.306397,5.639731,69.865320,53.935185,3.945707,0.641835


In [14]:
#auto_mode_starting_index_ins_row_values.iloc[0]['TimeStamp']

In [29]:
#overnight_auto_df.to_csv("Overnight_na.csv")